## House Price Prediction with MLFLOW


In [13]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [14]:
## Preparing the dataset
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data['Price'] = housing.target
data.head()




,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [15]:
print(data.columns.tolist())


['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'Price']


In [16]:
from urllib.parse import urlparse
## independent and dependent variables
X=data.drop(columns=['Price'])
y=data['Price']

In [17]:
## Hyperparameter tuning using GridSearchCV
def hyperparameter_tuining(X_train,y_train,param_grid):
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2,scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    return grid_search


In [19]:
from urllib.parse import urlparse
from mlflow.models import infer_signature
from sklearn.metrics import mean_squared_error

## Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Define the hyperparameters for the Random Forest Regressor
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

## Set tracking URI BEFORE starting the run
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

## Set experiment name
mlflow.set_experiment("California_Housing_Prediction")

## Start the MLflow experiment 
with mlflow.start_run():
    ## Perform hyperparameter tuning
    grid_search = hyperparameter_tuining(X_train, y_train, param_grid)
    
    ## Get the best model 
    best_model = grid_search.best_estimator_
    
    ## Evaluate the best model 
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    ## Log the best parameters ONE BY ONE
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    
    ## Log the metric
    mlflow.log_metric("mse", mse)
    
    ## Infer signature
    signature = infer_signature(X_train, best_model.predict(X_train))
    
    ## Log the model with signature and register it
    mlflow.sklearn.log_model(
        sk_model=best_model, 
        artifact_path="random_forest_model",
        signature=signature,
        registered_model_name="Best_Random_Forest_California"
    )
    
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Mean Squared Error: {mse}")


2026/02/12 15:41:18 INFO mlflow.tracking.fluent: Experiment with name 'California_Housing_Prediction' does not exist. Creating a new experiment.


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.0s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   2.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.9s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.1s
[CV] END max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.2s
[CV] END max_depth=5, min_samples_leaf=

2026/02/12 15:42:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/piyushgaidhani/Desktop/MLFlow/.venv/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Successfully registered model 'Best_Random_Forest_California'.
2026/02/12 15:42:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Best_Random_Forest_California, version 1


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Mean Squared Error: 0.25392349712045054
🏃 View run nosy-bug-308 at: http://127.0.0.1:5000/#/experiments/250786399259771127/runs/70426837cee844ae975428f23e391bd1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/250786399259771127


Created version '1' of model 'Best_Random_Forest_California'.
